# 🧩 Mini-Lab: Token Probabilities

**Module 2: LLM Core Concepts** | **Duration: ~30 min** | **Type: Mini-Lab**

---

## Learning Objectives

By the end of this mini-lab, you will be able to:

1. **Understand** what log probabilities (logprobs) represent
2. **Access** token-level probabilities from OpenAI API
3. **Analyze** model confidence for each generated token
4. **Use** logprobs for classification and uncertainty detection

## Target Concepts

| Concept | Description |
|---------|-------------|
| Log Probabilities | Natural log of token probability, measuring model confidence |
| Tokenization | Converting text to tokens (prerequisite) |

## Prerequisites

- **mini-tokenizer**: Understanding of tokens

## 1. Setup

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
import math
from IPython.display import Markdown, display

load_dotenv()
client = OpenAI()

def md(text):
    display(Markdown(text))

print("✓ Setup complete")

✓ Setup complete


## 2. Understanding Log Probabilities

**Log probability (logprob)** = natural log of probability

| Probability | Log Probability | Interpretation |
|-------------|-----------------|----------------|
| 100% (1.0) | 0.0 | Certain |
| 50% (0.5) | -0.69 | Uncertain |
| 10% (0.1) | -2.30 | Unlikely |
| 1% (0.01) | -4.61 | Very unlikely |

**Why log probabilities?**
- Numerical stability (avoid underflow)
- Addition instead of multiplication for sequences
- Standard in ML/statistics

In [2]:
def prob_to_logprob(p):
    """Convert probability to log probability."""
    return math.log(p)

def logprob_to_prob(lp):
    """Convert log probability to probability."""
    return math.exp(lp)

# Interactive conversion
print("📊 Probability ↔ Log Probability Conversion")
print("="*50)

probabilities = [0.99, 0.90, 0.75, 0.50, 0.25, 0.10, 0.01]
for p in probabilities:
    lp = prob_to_logprob(p)
    confidence = "🟢 High" if p > 0.8 else "🟡 Medium" if p > 0.5 else "🔴 Low"
    print(f"P = {p:.2%}  →  logprob = {lp:.3f}  ({confidence})")

📊 Probability ↔ Log Probability Conversion
P = 99.00%  →  logprob = -0.010  (🟢 High)
P = 90.00%  →  logprob = -0.105  (🟢 High)
P = 75.00%  →  logprob = -0.288  (🟡 Medium)
P = 50.00%  →  logprob = -0.693  (🔴 Low)
P = 25.00%  →  logprob = -1.386  (🔴 Low)
P = 10.00%  →  logprob = -2.303  (🔴 Low)
P = 1.00%  →  logprob = -4.605  (🔴 Low)


## 3. Getting Logprobs from OpenAI API

Use `logprobs=True` to get probability information for each token:

In [3]:
def get_response_with_logprobs(prompt, max_tokens=50):
    """Get response with token-level log probabilities."""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0,
        logprobs=True,
        top_logprobs=5  # Get top 5 alternatives for each position
    )
    
    return response

# Test
response = get_response_with_logprobs("The capital of France is")
print(f"Response: {response.choices[0].message.content}")
print(f"\nLogprobs available: {response.choices[0].logprobs is not None}")

Response: The capital of France is Paris.

Logprobs available: True


In [4]:
def visualize_logprobs(response):
    """Visualize token probabilities from response."""
    
    if not response.choices[0].logprobs:
        print("No logprobs in response!")
        return
    
    content = response.choices[0].logprobs.content
    
    print("\n📊 Token-by-Token Analysis")
    print("="*70)
    
    for i, token_data in enumerate(content):
        token = token_data.token
        logprob = token_data.logprob
        prob = logprob_to_prob(logprob)
        
        # Confidence indicator
        if prob > 0.95:
            conf = "🟢 Certain"
        elif prob > 0.75:
            conf = "🟡 Confident"
        elif prob > 0.50:
            conf = "🟠 Moderate"
        else:
            conf = "🔴 Uncertain"
        
        # Visual bar
        bar = "█" * int(prob * 20)
        
        # Display token (escape special chars)
        display_token = token.replace('\n', '\\n').replace('\t', '\\t')
        
        print(f"\n[{i}] '{display_token}'")
        print(f"    Prob: {prob:6.2%} (logprob: {logprob:.3f}) {bar} {conf}")
        
        # Show top alternatives
        if token_data.top_logprobs:
            print(f"    Alternatives:")
            for alt in token_data.top_logprobs[:3]:
                alt_prob = logprob_to_prob(alt.logprob)
                alt_token = alt.token.replace('\n', '\\n')
                print(f"      '{alt_token}': {alt_prob:.2%}")

visualize_logprobs(response)


📊 Token-by-Token Analysis

[0] 'The'
    Prob: 99.98% (logprob: -0.000) ███████████████████ 🟢 Certain
    Alternatives:
      'The': 99.98%
      'Paris': 0.01%
      'the': 0.00%

[1] ' capital'
    Prob: 100.00% (logprob: 0.000) ████████████████████ 🟢 Certain
    Alternatives:
      ' capital': 100.00%
      'capital': 0.00%
      ' Capital': 0.00%

[2] ' of'
    Prob: 100.00% (logprob: 0.000) ████████████████████ 🟢 Certain
    Alternatives:
      ' of': 100.00%
      ' city': 0.00%
      'of': 0.00%

[3] ' France'
    Prob: 100.00% (logprob: 0.000) ████████████████████ 🟢 Certain
    Alternatives:
      ' France': 100.00%
      'France': 0.00%
      ' Paris': 0.00%

[4] ' is'
    Prob: 100.00% (logprob: 0.000) ████████████████████ 🟢 Certain
    Alternatives:
      ' is': 100.00%
      ' هو': 0.00%
      'is': 0.00%

[5] ' Paris'
    Prob: 100.00% (logprob: -0.000) ███████████████████ 🟢 Certain
    Alternatives:
      ' Paris': 100.00%
      'Paris': 0.00%
      ' Пари': 0.00%

[6] '

## 4. Analyzing Model Confidence

Use logprobs to understand when the model is confident vs uncertain:

In [6]:
def analyze_confidence(prompt):
    """Analyze model confidence across the response."""
    
    response = get_response_with_logprobs(prompt, max_tokens=30)
    content = response.choices[0].logprobs.content
    
    tokens = []
    probs = []
    
    for token_data in content:
        tokens.append(token_data.token)
        probs.append(logprob_to_prob(token_data.logprob))
    
    avg_prob = np.mean(probs)
    min_prob = np.min(probs)
    min_idx = np.argmin(probs)
    
    print(f"\n📝 Prompt: {prompt}")
    print(f"📤 Response: {response.choices[0].message.content}")
    print(f"\n📊 Confidence Statistics:")
    print(f"   Average confidence: {avg_prob:.1%}")
    print(f"   Minimum confidence: {min_prob:.1%} at token '{tokens[min_idx]}'")
    print(f"   Total tokens: {len(tokens)}")
    
    # Identify uncertain tokens
    uncertain = [(i, t, p) for i, (t, p) in enumerate(zip(tokens, probs)) if p < 0.5]
    
    if uncertain:
        print(f"\n⚠️ Uncertain tokens (< 50% confidence):")
        for idx, token, prob in uncertain:
            print(f"   [{idx}] '{token}': {prob:.1%}")
    else:
        print(f"\n✅ All tokens above 50% confidence")
    
    return probs

# Compare factual vs creative prompts
print("\n" + "="*70)
print("FACTUAL PROMPT")
print("="*70)
analyze_confidence("What is 2 + 2?")

print("\n" + "="*70)
print("CREATIVE PROMPT")
print("="*70)
analyze_confidence("Write a creative word that describes happiness:")


FACTUAL PROMPT

📝 Prompt: What is 2 + 2?
📤 Response: 2 + 2 equals 4.

📊 Confidence Statistics:
   Average confidence: 99.9%
   Minimum confidence: 99.1% at token ' equals'
   Total tokens: 8

✅ All tokens above 50% confidence

CREATIVE PROMPT

📝 Prompt: Write a creative word that describes happiness:
📤 Response: "Joyglow" - a radiant state of happiness that illuminates one's spirit and brightens the world around them.

📊 Confidence Statistics:
   Average confidence: 77.2%
   Minimum confidence: 27.4% at token 'gl'
   Total tokens: 24

⚠️ Uncertain tokens (< 50% confidence):
   [2] 'gl': 27.4%
   [5] ' -': 43.1%
   [7] ' radiant': 46.8%
   [8] ' state': 35.7%
   [12] ' illumin': 39.7%
   [14] ' one's': 34.1%
   [17] ' bright': 31.9%


[0.611696846736773,
 0.9819413556270229,
 0.2741468890900173,
 0.9912431846933826,
 0.9997252355748864,
 0.4311681117146683,
 0.927823599740573,
 0.4683588870072995,
 0.3569191407471089,
 0.9990793160283619,
 0.9308707953959569,
 0.9989319161051103,
 0.3974039176757734,
 1.0,
 0.34078324763253887,
 0.8061138397206027,
 0.9397901664200573,
 0.3192055297680872,
 1.0,
 0.9996458892352142,
 0.7737587027519198,
 0.9999932502087602,
 0.9852807830011578,
 0.9875489701375568]

## 5. Using Logprobs for Classification

A powerful use case: get classification confidence without generating text:

In [7]:
def classify_with_confidence(text, categories):
    """Classify text and get confidence from logprobs."""
    
    # Format categories for prompt
    cat_str = "/".join(categories)
    
    prompt = f"""Classify the following text into one category: {cat_str}

Text: "{text}"

Category (respond with only the category name):"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=10,
        temperature=0,
        logprobs=True,
        top_logprobs=5
    )
    
    # Get the first meaningful token
    predicted = response.choices[0].message.content.strip()
    first_token = response.choices[0].logprobs.content[0]
    confidence = logprob_to_prob(first_token.logprob)
    
    # Get alternative classifications
    alternatives = []
    for alt in first_token.top_logprobs:
        alt_text = alt.token.strip().lower()
        for cat in categories:
            if alt_text in cat.lower():
                alternatives.append((cat, logprob_to_prob(alt.logprob)))
                break
    
    return {
        "text": text,
        "prediction": predicted,
        "confidence": confidence,
        "alternatives": alternatives
    }

# Test sentiment classification
categories = ["Positive", "Negative", "Neutral"]
test_texts = [
    "This product is absolutely amazing! Best purchase ever!",
    "Terrible experience. Would not recommend.",
    "The package arrived on time.",
    "It's okay, nothing special but not bad either.",
]

print("\n🏷️ Sentiment Classification with Confidence")
print("="*60)

for text in test_texts:
    result = classify_with_confidence(text, categories)
    
    conf_bar = "█" * int(result['confidence'] * 20)
    conf_level = "🟢" if result['confidence'] > 0.9 else "🟡" if result['confidence'] > 0.7 else "🔴"
    
    print(f"\n📝 \"{text[:50]}...\"")
    print(f"   Prediction: {result['prediction']}")
    print(f"   Confidence: {result['confidence']:.1%} {conf_bar} {conf_level}")


🏷️ Sentiment Classification with Confidence

📝 "This product is absolutely amazing! Best purchase ..."
   Prediction: Positive
   Confidence: 100.0% ███████████████████ 🟢

📝 "Terrible experience. Would not recommend...."
   Prediction: Negative
   Confidence: 100.0% ███████████████████ 🟢

📝 "The package arrived on time...."
   Prediction: Positive
   Confidence: 100.0% ███████████████████ 🟢

📝 "It's okay, nothing special but not bad either...."
   Prediction: Neutral
   Confidence: 100.0% ███████████████████ 🟢


## 6. Perplexity: Measuring Overall Confidence

**Perplexity** measures how "surprised" the model is by a sequence:

In [8]:
def calculate_perplexity(logprobs):
    """Calculate perplexity from log probabilities."""
    # Perplexity = exp(-average_logprob)
    avg_logprob = np.mean(logprobs)
    perplexity = math.exp(-avg_logprob)
    return perplexity

def analyze_perplexity(prompt):
    """Analyze response perplexity."""
    
    response = get_response_with_logprobs(prompt, max_tokens=50)
    content = response.choices[0].logprobs.content
    
    logprobs = [t.logprob for t in content]
    perplexity = calculate_perplexity(logprobs)
    
    print(f"\n📝 Prompt: {prompt}")
    print(f"📤 Response: {response.choices[0].message.content[:100]}...")
    print(f"\n📊 Perplexity: {perplexity:.2f}")
    
    if perplexity < 2:
        print("   Interpretation: Very confident (common/predictable output)")
    elif perplexity < 5:
        print("   Interpretation: Moderately confident")
    else:
        print("   Interpretation: Uncertain (uncommon/creative output)")
    
    return perplexity

# Compare different types of prompts
print("\n" + "="*60)
print("PERPLEXITY COMPARISON")
print("="*60)

prompts = [
    "What is 1 + 1?",  # Should be very confident
    "Explain quantum computing briefly.",  # Technical but known
    "Write a haiku about a purple elephant.",  # Creative/uncertain
]

for prompt in prompts:
    analyze_perplexity(prompt)


PERPLEXITY COMPARISON

📝 Prompt: What is 1 + 1?
📤 Response: 1 + 1 equals 2....

📊 Perplexity: 1.00
   Interpretation: Very confident (common/predictable output)

📝 Prompt: Explain quantum computing briefly.
📤 Response: Quantum computing is a type of computation that leverages the principles of quantum mechanics to pro...

📊 Perplexity: 1.09
   Interpretation: Very confident (common/predictable output)

📝 Prompt: Write a haiku about a purple elephant.
📤 Response: In twilight's embrace,  
A purple elephant roams,  
Dreams in gentle steps....

📊 Perplexity: 1.38
   Interpretation: Very confident (common/predictable output)


## 7. Practical Application: Hallucination Detection

Low-confidence tokens might indicate potential hallucinations:

In [ ]:
def detect_potential_hallucinations(prompt, confidence_threshold=0.3):
    """Identify potentially hallucinated content based on low confidence."""
    
    response = get_response_with_logprobs(prompt, max_tokens=100)
    content = response.choices[0].logprobs.content
    
    full_response = response.choices[0].message.content
    
    # Find low-confidence regions
    suspicious = []
    for i, token_data in enumerate(content):
        prob = logprob_to_prob(token_data.logprob)
        if prob < confidence_threshold:
            # Get surrounding context
            start = max(0, i - 2)
            end = min(len(content), i + 3)
            context = "".join([content[j].token for j in range(start, end)])
            suspicious.append({
                "token": token_data.token,
                "probability": prob,
                "context": context,
                "position": i
            })
    
    print(f"\n🔍 Hallucination Detection Analysis")
    print("="*60)
    print(f"📝 Prompt: {prompt}")
    print(f"📤 Response: {full_response}")
    print(f"\n📊 Threshold: {confidence_threshold:.0%} confidence")
    
    if suspicious:
        print(f"\n⚠️ Found {len(suspicious)} potentially uncertain claims:")
        for item in suspicious[:5]:  # Show top 5
            print(f"\n   Token: '{item['token']}'")
            print(f"   Confidence: {item['probability']:.1%}")
            print(f"   Context: '...{item['context']}...'")
    else:
        print(f"\n✅ No tokens below {confidence_threshold:.0%} confidence")
    
    return suspicious

# Test with a prompt that might cause hallucinations
detect_potential_hallucinations(
    "tell me how many oceans are living on the moon"
)


🔍 Hallucination Detection Analysis
📝 Prompt: tell me how many oceans are living on the moon
📤 Response: The Moon does not have any oceans like Earth does. While there are features on the Moon's surface that resemble large bodies of water, such as the dark basaltic plains known as "maria," these are not oceans but rather solidified lava flows. The Moon has no liquid water bodies, and any water present is primarily in the form of ice, found in permanently shadowed craters at the poles.

📊 Threshold: 30% confidence

⚠️ Found 1 potentially uncertain claims:

   Token: ' large'
   Confidence: 24.1%
   Context: '... that resemble large bodies of...'


[{'token': ' large',
  'probability': 0.24137756356103526,
  'context': ' that resemble large bodies of',
  'position': 22}]

: 

## 🎯 Summary

### Key Takeaways

1. **Log Probabilities Basics**
   - logprob = ln(probability)
   - 0 = certain, more negative = less confident
   - Convert: prob = e^logprob

2. **Accessing Logprobs**
   - Use `logprobs=True` in API call
   - `top_logprobs=N` for alternatives
   - Available per-token in response

3. **Use Cases**
   - **Confidence measurement**: Know when model is uncertain
   - **Classification**: Get probability for each class
   - **Hallucination detection**: Flag low-confidence claims
   - **Quality filtering**: Reject low-confidence outputs

4. **Perplexity**
   - Aggregate measure of sequence confidence
   - Lower = more predictable/confident
   - Useful for comparing outputs

### Next Steps

- **mini-streaming**: Real-time token delivery
- **mini-model-compare**: Compare models using logprobs
- **Module 8**: Use logprobs in evaluation pipelines